In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size= 128
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [ ]:
# datasets
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# model class
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ = self.rnn(x, h0)
    out=  out[:, -1, :]
    out=  self.fc(out)
    return out



In [ ]:
model= RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
crieterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = crieterion(outputs, labels)

    # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%10==0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [10/600], Loss: 2.1492
Epoch [1/2], Step [20/600], Loss: 1.7614
Epoch [1/2], Step [30/600], Loss: 1.4831
Epoch [1/2], Step [40/600], Loss: 1.3096
Epoch [1/2], Step [50/600], Loss: 1.2199
Epoch [1/2], Step [60/600], Loss: 1.1492
Epoch [1/2], Step [70/600], Loss: 0.9703
Epoch [1/2], Step [80/600], Loss: 1.1338
Epoch [1/2], Step [90/600], Loss: 0.7897
Epoch [1/2], Step [100/600], Loss: 0.8115
Epoch [1/2], Step [110/600], Loss: 0.7216
Epoch [1/2], Step [120/600], Loss: 0.9243
Epoch [1/2], Step [130/600], Loss: 0.6988
Epoch [1/2], Step [140/600], Loss: 0.9381
Epoch [1/2], Step [150/600], Loss: 0.8351
Epoch [1/2], Step [160/600], Loss: 0.6626
Epoch [1/2], Step [170/600], Loss: 0.7633
Epoch [1/2], Step [180/600], Loss: 0.7625
Epoch [1/2], Step [190/600], Loss: 0.5900
Epoch [1/2], Step [200/600], Loss: 0.8431
Epoch [1/2], Step [210/600], Loss: 0.5948
Epoch [1/2], Step [220/600], Loss: 0.7990
Epoch [1/2], Step [230/600], Loss: 0.8173
Epoch [1/2], Step [240/600], Loss: 0.6408
E

In [ ]:
# Test the model
with torch.no_grad():
  n_correct=0
  n_samples=0
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)
    # max returns(value, index)
    _, predicted = torch.max(outputs.data, 1)
    n_samples += labels.size(0)
    n_correct += (predicted==labels).sum().item()

acc = 100.0*n_correct/n_samples
print(f'Accuracy of the model on the 10000 test images: {acc} %')

Accuracy of the model on the 10000 test images: 94.25 %
